In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


In [4]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [5]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            100.67799289874142,
            6.339057689337892
          ],
          [
            100.67596196772081,
            6.3382887311459655
          ],
          [
            100.67522051671278,
            6.336943051553064
          ],
          [
            100.67264155668596,
            6.326561976772766
          ],
          [
            100.6766711817279,
            6.325216266618256
          ],
          [
            100.67841197974587,
            6.326369732679765
          ],
          [
            100.68079751777259,
            6.326241569910792
          ],
          [
            100.68086199177264,
            6.327010546046168
          ],
          [
            100.68150673177979,
            6.327491155548643
          ],
          [
            100.68131330977798,
            6.330374803178657
          ],
          [
            100.68095870277352,
            6.331047651975837
          ],
          [
            100.68150673177979,
            6.332201105017177
          ],
          [
            100.68260278979051,
            6.331496217353248
          ],
          [
            100.68366661080216,
            6.331303975096517
          ],
          [
            100.684601483812,
            6.331848661305088
          ],
          [
            100.6842468768092,
            6.332681709689936
          ],
          [
            100.68518174981904,
            6.3354051277208185
          ],
          [
            100.68421463980837,
            6.338160571336402
          ],
          [
            100.6780251357423,
            6.339057689337892
          ]
        ],
        "type": "LineString"
      }
    }
  ]
}

In [47]:
coords = geoJSON['features'][0]\
                ['geometry']\
                ['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [82]:
collection = ee.Image(ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
              .filterBounds(aoi) \
              .filterDate('2020-01-01', '2020-12-31') \
              .first()\
              .clip(aoi))

In [83]:
collection.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60']

In [101]:
import IPython.display as disp

url = collection.select('B12').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

In [110]:
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(collection.select('B8'),
                   collection.select('B12'),
                   collection.select('B11').divide(collection.select('B8')))


# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 1]}, 'NDVI Overlay')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)